# 1. Notes

## 1.1 Remarks
#### DP as a cache
- Try to use prefix, cached `array`, `matrix`, `dictionary` to reduce time complexity
- `dp[i]`: represent the `prefix optimal value` in the segment `A[:i]`
- `dp[i][j]`: represent the `prefix optimal value` in the square `AB[:i,:j]`

#### Allocate additional mem

- Avoid accessing restricted heap mem

```Cpp
vector<vector<int>> dp(n+10, vector<int>(m+10, NULL)); 
```

- Bottom up DP

```Cpp
for(int i=0; i<=n; ++i) for(int j=0; j<=m; ++j)
    if(i<n)
    if(j<m)
```

## 1.2 recursion/backtrack return values
- Normal return

```C++
// Return when dfs only generate 1 case
return dfs(x, str, i + 1)
```

- Search - backtrack

```C++
// [OR] Search for the true case
//     if >= 1 case true -> true
//     if all cases false -> false
// Early stopping usage: if 1 case true -> early stop
FOR(x,0,4)
    if(dfs(x, str, i + 1)) return true;
```

## 1.3 Variable in recursion/backtrack (Important)
- If we change the variable before calling recursion, remember to change it back after calling

```Cpp
// Change back after recur if modify current var (--, ++)
void dfs(int i, int a, int b) {
    a += 1;
    b -= 1;
    dfs(i+1, a, b);
    a -= 1;
    b += 1;
}

// Equivalent, Need not change back after backtracking
void dfs(int i, int a, int b) {
    dfs(i+1, a+1, b-1);
}
```

#### Example

```Cpp
// Remember to change back color_1, color_2 values
if(b > 0) color_1 += 1; 
if(A[k] - b > 0) color_2 += 1;
dfs(k+1, box_1, num_1 + b, color_1, box_2, num_2 + A[k]-b, color_2);
if(b > 0) color_1 -= 1; 
if(A[k] - b > 0) color_2 -= 1;
```



# 2. DP
- Think about transition from state `j` -> state `i`
    + How `state i` is built from `state j`

<img src="./img/3.jpg" width="600"/>


```C++
// State j --> state i (j < i)
dp[i] = dp[j] + 1;
dp[i] = dp[i-1] + 2; // j=i-1

// Example: state j --> state i (j < i)
for(int i=1; i<N; ++i) {
    for(int j=0; j<i; ++j) {
        // Full relaxation condition
        if(dp[i] < dp[j] + A[i]) {
            // Relax
            dp[i] = A[i] + dp[j];

            // backtrack
            backtrack[i] = j;
        }

        // Relaxation Short-form
        dp[i] = max(dp[i], dp[j] + A[i]);
    }
}
```

## 2.1 Push (Bottom Up) vs Pull DP (Top Down)
#### Push DP
- Main pointer = state `j` (Propagate `j` -> `i`)
- Don't need to worry about the order, jush push then retrieve later

```C++
// Push DP - Bottom Up
for each state j:
  for each state i that affected by j:
    dp[i] = dp[j] + ...

// Example: Push to i=j+1, i=j+2
dp[0] = 1;
for(int j=0; j<N; ++j) {
    if(j+1 <= N) dp[j+1] += dp[j];
    if(j+2 <= N) dp[j+2] += dp[j];
}
```

#### Pull DP
- Main pointer = state `i` (Propagate `i` <- `j`s)
- All state `j`s must be computed before being pulled by state `i`

```C++           
// Pull DP - Top Down
for each state i:
  for each state j that contributes:
    dp[i] = dp[j] + ...

// Example: Pull from j=i-1, j=i-2
dp[0] = 1;
for(int i=1; i<=N; ++i) {
    int cur = 0;
    if(i-1 >= 0) cur += dp[i-1];
    if(i-2 >= 0) cur += dp[i-2];
    dp[i] = cur;
}
```

## 2.2 Loop vs Recursive
#### Recursive
+ Easy to implement, natural approach
+ Not optimized in runtime (mostly complete search)
    + Try applying **pruning + caching** to reduce time complexity
+ Can implement both: push and pull DP
+ **Recursive = DFS**
+ Condition
    + Don't need to worry about the order
    + If relaxing `i`, dfs will recursively computes all needed `j`
    + Cache all computations
    

#### Loop
+ harder implementation
+ More optimal
+ Can implement both: push and pull DP
+ **Loop = BFS** (Propagate with order and levels)
    + In some cases, Consider convert loop -> bfs to optimize/prune branches
+ Condition
    + Need to think about order of propagation
    + DP relaxed in 1 order (i.e increasing, left->right)
+ Common Propagation order:
    + Left -> Right (Prefix), Right -> Left (Suffix)
    + By length (substring), By area
    + Guide by a dfs, bfs, or priority_queue

# 3. Backtracking
#### Backtracking
<img src="./img/9.jpg" alt="drawing" width="400"/>

#### Backtracking - remove tail recursion
<img src="./img/10.jpg" alt="drawing" width="400"/>

#### DFS
<img src="./img/11.jpg" alt="drawing" width="400"/>


# 4. Templates
#### 4.1 Backtrack DP template

```C++
bool get(int i) {
    // base cases + prunes
    if(prunes) return false;
    if(base_cases) return x;
    
    // End
    if(success_cond) {
        print(ans);
        return true;
    }

    // Check visited or dp
    vis[i] = true;

    // Branching
    next_ = ...+...;
    if(0<=next_ && next_< N && vis[next_] == false) {
        ans += ...;
        if(get(next_)) return true;
        ans -= ...;
    }

    next_ = ...-...;
    if(0<=next_ && next_< N && vis[next_] == false) {
        ans += ...;
        if(get(next_)) return true;
        ans -= ...;
    }

    for() {
        next_ = ;
        if(0<=next_ && next_< N && vis[next_] == false) {
            ans += ...;
            if(get(next_)) return true;
            ans -= ...;
        }
    }

    return false;
}
```

#### 4.2 Recursive cached template
- cache dp before return

```C++
T dp[];

T get(int i) {
    // base cases
    if (i == 0) return 1;

    // cached return
    if(dp[i] != NULL) return dp[i];
    
    // Branching past stages
    if(cond1) {
        // Cache
        T res = get(j);
        dp[j] = res;
        return res;
    }
    
    for() {
        if(cond2) {
            // Cache
            T res = get(k);
            dp[k] = res;
            return res;
        }
    }

    // relax cur stage from i+1
    T res = get(i+1);
    dp[i] = res + ...;
    return dp[i];
}
```

#### 4.3 Accumulate DP Template

```C++
T get(int i) {
    // base case
    if (i == 0) return 1;
    
    // cached return
    if(dp[i] != NULL) return dp[i];
    
    // Branching
    int cur = 0;
    
    if(cond1) cur += get(j);
    for() {
        if(cond2) {
            cur += get(j);
        }
    }
    
    // Relax
    dp[i] = cur;
    return cur;
}
```

#### 4.4 Multiple cond DP Template

```C++
for(i in {0,N})
    // Case 1:
    if()
        dp[i] = dp[j] + 1

    // Case 2:
    else if()
        dp[i] = dp[i-1] + 2

    // Case for:
    else for() if()
        dp[i] = dp[i-2] + 3
```

#### 4.5 Propagate state

```C++
for(i in {0,N})
    // Case exist i from j
    if(exist.count(i))
        dp[i] = dp[j] + 1

    // Case not exist i or not choose i --> propagate
    else
        dp[i] = dp[i-1]
```